In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:

Write a function to normalize features
Implement coordinate descent for LASSO
Explore effects of L1 penalty

In [1]:
import numpy as np
import pandas as pd

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 
              'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 
              'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 
              'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

In [3]:
sales=pd.read_csv('kc_house_data.csv', dtype=dtype_dict)
train=pd.read_csv("kc_house_train_data.csv", dtype=dtype_dict)
test=pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict)

In [4]:
def get_data(data_frame, features, output):
    '''args: 
    data_frame= array, dataframe alike
    features: list-like []
    output: str'''
    
    import numpy as np
    
    data_frame["constant"]=1
    features= ["constant"] + features
    
    features_frame= data_frame[features]
    feature_matrix= np.array(features_frame)
    
    output_array = np.array(data_frame[output])
    return(feature_matrix, output_array)

In [5]:
def predict_output(feature_matrix, weights):
    
    predictions = np.dot(feature_matrix, weights)
    return predictions

In [6]:
def normalize_features(feature_matrix):
    norms=np.linalg.norm(feature_matrix, axis=0)
    X_normalized = feature_matrix/norms
    return X_normalized, norms

# 
- cost function: SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)

- coordinate descent: argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]

- optimizing each coordinate:        

       ┌ (ro[i] + lambda/2)    if ro[i] < -lambda/2
w[i] = ├ 0 _____________________                         if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)    if ro[i] > lambda/2
       
- ro func: ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].

In [7]:
simple_features=['sqft_living','bedrooms']
target= 'price'

simple_features_matrix, output = get_data(sales,simple_features,target)
simple_features_matrix, norms  = normalize_features(simple_features_matrix)

initial_weights= np.array([1.,4.,1.])

simple_predictions = predict_output(simple_features_matrix, initial_weights)

In [8]:
#compute values for ro[i]

ro  = np.zeros(simple_features_matrix.shape[1]) #init ro values with zeros
foo = (output - simple_predictions) + np.dot(initial_weights,simple_features_matrix.T)
ro  = np.dot(simple_features_matrix.T, foo)

In [9]:
ro

array([79400304.6376446 , 87939472.68182787, 80966703.40538491])

In [10]:
initial_weights.shape

(3,)

In [11]:
simple_features_matrix.shape

(21613, 3)

In [12]:
foo = (output - simple_predictions)  + np.dot(initial_weights,simple_features_matrix.T)
ro = np.dot(simple_features_matrix.T, foo)
ro

array([79400304.6376446 , 87939472.68182787, 80966703.40538491])

# Q1  Recall that, whenever ro[i] falls between -l1_penalty/2 and l1_penalty/2, the corresponding weight w[i] is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of l1_penalty would not set w[1] zero, but would set w[2] to zero, if we were to take a step in that coordinate?

w[2] = 0  if -lambda/2 <= ro[2] <= lambda/2

lambda >= 2ro[2] and lambda >= -2ro[2]


In [13]:
2*ro[2], 2*ro[1]

(161933406.81076983, 175878945.36365575)

lambda has to be greater th`an ro[2] but less than ro[1] 

# Q2 lambda values for both Ws ot be zero

lambda > 2*ro[1]

# single coordinate descent step

In [14]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    
    prediction= predict_output(feature_matrix, weights)
    
    foo = (output - prediction)  + np.dot(weights[i], feature_matrix[:,i])
    ro = np.dot(feature_matrix[:,i], foo)
    
    if i==0: #is intercept
        new_weight_i = ro
    elif ro < -0.5*l1_penalty:
        new_weight_i = ro + 0.5*l1_penalty
    elif ro > 0.5*l1_penalty:
        new_weight_i = ro - 0.5*l1_penalty
    else:
        new_weight_i = 0
        
    return new_weight_i

In [15]:
# test
import math 
feat=np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]])
output=np.array([1., 1.])
weights=np.array([1., 4.])
print(lasso_coordinate_descent_step(1, feat, output , weights , 0.1))

0.4255588466910251


# Cyclical coordinate descent


Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

In [16]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    
    converged = False
    weights = np.array(initial_weights)
    delta = np.zeros(weights.shape)
    D = feature_matrix.shape[1]
    
    while not converged:
        
        for i in range(D):
            weight_update = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            delta[i] = np.abs(weight_update - weights[i])
            weights[i] = weight_update
            
        max_delta = max(delta)
        if max_delta < tolerance:
            converged = True
    
    return weights

In [17]:
# try out simple model
simple_features=['sqft_living','bedrooms']
target= 'price'

simple_features_matrix, output = get_data(sales,simple_features,target)
normalized_simple_features_matrix, norms  = normalize_features(simple_features_matrix)
initial_weights=np.zeros(simple_features_matrix.shape[1])
l1=1e7
tolerance =1.0

In [18]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_features_matrix,output, initial_weights,l1,tolerance)

In [19]:
#lean on whole sales set

prediction = predict_output(normalized_simple_features_matrix, weights)

In [20]:
rss = ( (output-prediction)**2 ).sum()

In [21]:
print('rss is {:.3e}'.format(rss))

rss is 1.630e+15


In [22]:
weights

array([21624997.95951914, 63157247.20788951,        0.        ])

# lasso with more features

In [30]:
def table_weights(features_list, weights):
    featuress=list(features_list)

    featuress.insert(0,'constant')
    df=pd.DataFrame(data={'Features':featuress, 'Weights':weights})
    return df

In [24]:
features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
            'waterfront', 'view','condition', 'grade', 'sqft_above',
            'sqft_basement', 'yr_built', 'yr_renovated']

target = 'price'

len(features)

13

In [26]:
feature_matrix, y_train = get_data(train, features, target)
normalized_feature_matrix, norms = normalize_features(feature_matrix)

l1_penalty=1e7
initial_weights = np.zeros(normalized_feature_matrix.shape[1])
tolerance =1.

weights1e7=lasso_cyclical_coordinate_descent(normalized_feature_matrix, y_train, initial_weights,
                                            l1_penalty,tolerance)


## Q5 In the model trained with l1_penalty=1e7//1e8, which of the following features has non-zero weight? (Select all that apply)

In [31]:
table_weights(features, weights1e7)

,Features,Weights
0,constant,2.442960e+07
1,bedrooms,0.000000e+00
2,bathrooms,0.000000e+00
3,sqft_living,4.838917e+07
4,sqft_lot,0.000000e+00
5,floors,0.000000e+00
6,waterfront,3.317511e+06
7,view,7.329962e+06
8,condition,0.000000e+00
9,grade,0.000000e+00


In [32]:
feature_matrix, y_train = get_data(train, features, target)
normalized_feature_matrix, norms = normalize_features(feature_matrix)

l1_penalty=1e8
initial_weights = np.zeros(normalized_feature_matrix.shape[1])
tolerance =1.

weights1e8=lasso_cyclical_coordinate_descent(normalized_feature_matrix, y_train, initial_weights,
                                            l1_penalty,tolerance)

table_weights(features, weights1e8)

,Features,Weights
0,constant,7.111463e+07
1,bedrooms,0.000000e+00
2,bathrooms,0.000000e+00
3,sqft_living,0.000000e+00
4,sqft_lot,0.000000e+00
5,floors,0.000000e+00
6,waterfront,0.000000e+00
7,view,0.000000e+00
8,condition,0.000000e+00
9,grade,0.000000e+00


In [33]:
feature_matrix, y_train = get_data(train, features, target)
normalized_feature_matrix, norms = normalize_features(feature_matrix)

l1_penalty=1e4
initial_weights = np.zeros(normalized_feature_matrix.shape[1])
tolerance =5e5

weights1e4=lasso_cyclical_coordinate_descent(normalized_feature_matrix, y_train, initial_weights,
                                            l1_penalty,tolerance)

table_weights(features, weights1e4)

,Features,Weights
0,constant,7.856474e+07
1,bedrooms,-2.209740e+07
2,bathrooms,1.279107e+07
3,sqft_living,9.380809e+07
4,sqft_lot,-2.013173e+06
5,floors,-4.219185e+06
6,waterfront,6.482843e+06
7,view,7.127409e+06
8,condition,5.001665e+06
9,grade,1.432752e+07


# Rescaling learned weights


In [37]:
def weights_normalized(weights):
    normalized_weights = weights / norms
    return normalized_weights

In [40]:
norm_weights_1e7=weights_normalized(weights1e7)
norm_weights_1e8=weights_normalized(weights1e8)
norm_weights_1e4=weights_normalized(weights1e4)

In [42]:
# evaluate models on test data WITHOUT normalizing feature

In [43]:
test_feature_matrix, y_test = get_data(test, features, target)

In [47]:
prediction =  predict_output(test_feature_matrix, norm_weights_1e7)
RSS = ( (y_test-prediction)**2 ).sum()
print('RSS for model with weights1e7 ={:.3e} '.format( RSS))

RSS for model with weights1e7 =2.760e+14 


In [48]:
prediction =  predict_output(test_feature_matrix, norm_weights_1e8)
RSS = ( (y_test-prediction)**2 ).sum()
print('RSS for model with weights1e8 ={:.3e} '.format( RSS))

RSS for model with weights1e8 =5.372e+14 


In [49]:
prediction = predict_output(test_feature_matrix, norm_weights_1e4)
RSS = ( (y_test-prediction)**2 ).sum()
print('RSS for model with weights1e4 ={:.3e} '.format( RSS))

RSS for model with weights1e4 =2.285e+14 
